# Nats notebook

Follow instructions from https://docs.nats.io/running-a-nats-service/nats_docker to set up the Nats server.

Create a network
```
docker network create nats
```
Set up a server
```
docker run -p 4222:4222 -p 8222:8222 -p 6222:6222 --network nats --name nats-server -ti nats:<version> -js
```
Add your Jupyter docker container to the network. Instead of '<your_jupyter_container_name>>, you need to provide the actual name of your Jupyter notebook container. You can find it using docker ps command.
```
docker network connect nats <your_jupyter_container_name>
```
Install Python client on your notebook container
```
pip install nats-py
```
Follow examples from https://github.com/nats-io/nats.py/tree/main

## Getting started example

In [1]:
from nats import NATS
import asyncio
import nats
from nats.errors import ConnectionClosedError, TimeoutError, NoServersError

In [2]:
async def main():
    # It is very likely that the demo server will see traffic from clients other than yours.
    # To avoid this, start your own locally and modify the example to use it. 
    # Use your server container name
    nc = await nats.connect("nats://nats-server:4222")

    # You can also use the following for TLS against the demo server.
    #
    # nc = await nats.connect("tls://demo.nats.io:4443")

    async def message_handler(msg):
        subject = msg.subject
        reply = msg.reply
        data = msg.data.decode()
        print("Received a message on '{subject} {reply}': {data}".format(
            subject=subject, reply=reply, data=data))

    # Simple publisher and async subscriber via coroutine.
    sub = await nc.subscribe("foo", cb=message_handler)

    # Stop receiving after 2 messages.
    await sub.unsubscribe(limit=2)
    await nc.publish("foo", b'Hello')
    await nc.publish("foo", b'World')
    await nc.publish("foo", b'!!!!!')

    # Synchronous style with iterator also supported.
    sub = await nc.subscribe("bar")
    await nc.publish("bar", b'First')
    await nc.publish("bar", b'Second')

    try:
        async for msg in sub.messages:
            print(f"Received a message on '{msg.subject} {msg.reply}': {msg.data.decode()}")
            await sub.unsubscribe()
    except Exception as e:
        pass

    async def help_request(msg):
        print(f"Received a message on '{msg.subject} {msg.reply}': {msg.data.decode()}")
        await nc.publish(msg.reply, b'I can help')

    # Use queue named 'workers' for distributing requests
    # among subscribers.
    sub = await nc.subscribe("help", "workers", help_request)

    # Send a request and expect a single response
    # and trigger timeout if not faster than 500 ms.
    try:
        response = await nc.request("help", b'help me', timeout=0.5)
        print("Received response: {message}".format(
            message=response.data.decode()))
    except TimeoutError:
        print("Request timed out")

    # Remove interest in subscription.
    await sub.unsubscribe()

    # Terminate connection to NATS.
    await nc.drain()

In [3]:
await main()

Received a message on 'foo ': Hello
Received a message on 'foo ': World
Received a message on 'bar ': First
Received a message on 'bar ': Second
Received a message on 'help _INBOX.jldI3AbJSaeL1hTrWR1pbM.jldI3AbJSaeL1hTrWR1peka617': help me
Received response: I can help


## JetStream and key-value example

See 
https://github.com/nats-io/nats.py/blob/main/examples/kv.py

In [5]:
async def kv_main():
    nc = await nats.connect("nats://nats-server:4222")
    js = nc.jetstream()

    # Create a KV
    kv = await js.create_key_value(bucket="MY_KV")

    # Set and retrieve a value
    await kv.put("hello", b"world")
    await kv.put("goodbye", b"farewell")
    await kv.put("greetings", b"hi")
    await kv.put("greeting", b"hey")

    # Retrieve and print the value of 'hello'
    entry = await kv.get("hello")
    print(f"KeyValue.Entry: key={entry.key}, value={entry.value}")
    # KeyValue.Entry: key=hello, value=world

    # Retrieve keys with filters
    filtered_keys = await kv.keys(filters=["hello", "greet"])
    print(f"Filtered Keys: {filtered_keys}")
    # Expected Output: ['hello', 'greetings']

    await nc.close()

In [6]:
await kv_main()

KeyValue.Entry: key=hello, value=b'world'
Filtered Keys: ['hello', 'goodbye', 'greetings', 'greeting']
